#### Notes
1. Importing Spreadsheets or CSV files (QGIS3) - https://www.qgistutorials.com/en/docs/3/importing_spreadsheets_csv.html
2. Nearest Neighbor Analysis (QGIS3) - https://www.qgistutorials.com/en/docs/3/nearest_neighbor_analysis.html
3. Getting list of coordinates (lat,long) from OSMNX Geoseries - https://copyprogramming.com/howto/plotting-a-route-with-osmnx
4. QNEAT3 Algorithms - https://root676.github.io/OdMatrixAlgs
5. Source Paper - https://www.researchgate.net/publication/319291962_The_Traditional_Four_Steps_Transportation_Modeling_Using_Simplified_Transport_Network_A_Case_Study_of_Dhaka_City_Bangladesh

In [107]:
import osmnx as ox
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Steps in Generating Demand Matrix

In [44]:
# CSV File generated from QGIS
df_centroids_to_node = pd.read_csv("sample_barangay_centroids_to_nearest_node.csv")
#df_centroids_to_node

# For sample only, will delete later
df_centroids_to_node.drop(index=[6], inplace=True)
#df_centroids_to_node

df_cn = df_centroids_to_node[["Barangay Name","HubName"]]
df_cn.set_index('Barangay Name', inplace=True)
nodes = list(df_cn["HubName"].unique())
centroids = list(df_cn.index)
df_cn_final = pd.DataFrame(nodes, columns=["Node"])

list_all = []
for node in nodes:
    list_c = []
    for centroid in centroids:
        a = df_cn.loc[centroid] == node
        if a.iloc[0]:
            list_c.append(centroid)
    list_all.append(list_c)
    
df_cn_final["Centroids"] = list_all
df_cn_final.set_index("Node", inplace=True)
df_cn_final

,Centroids
Node,
Sta. Lucia Grandmall,[San Roque]
Bayan Bayanan,"[Concepcion Dos, Concepcion Uno]"
Ligaya,[Calumpang]
Riverbanks,[Barangka]
Marikina Bayan,"[Jesus de La Pena, Santo Nino, Santa Elena]"
Tumana,"[Tumana, Marikina Heights]"
Nangka,"[Fortune, Nangka]"
Fairlane,[Parang]
Golden Valley,[Malanday]


#### 9 Modal Split
#### 9.1 Utility Matrices 

Centroid and then to Node: Cost Matrix Tables (in terms of time and cost) using QGIS

Network Type: https://www.qgistutorials.com/en/docs/3/downloading_osm_data.html

Vector -> QuickOSM -> QuickOSM dialog box

Processing -> Toolbox -> QNEAT3 -> Distance Matrices -> OD-Matrix From Points as Table (n:n)

#### 9.1.a. Bus

'type'='route' and 'public_transport:version'='2'

In [45]:
# Travel Time Matrix, Travel Cost Matrix
# Bus, Car, Motorcycle

# bus: 15 pesos for the first 5 km, 2 pesos for every succeeding km

In [46]:
csv_bus_travel_dist_mat = pd.read_csv("bus_cost_matrix.csv")
csv_bus_travel_dist_mat.head()

,origin_id,destination_id,entry_cost,network_cost,exit_cost,total_cost
0,Barangka,Barangka,0.000000,0.000000,0.000000,0.000000
1,Barangka,Calumpang,0.000016,3216.298944,0.000454,3216.299414
2,Barangka,Concepcion Dos,0.000016,5861.484544,0.003020,5861.487580
3,Barangka,Concepcion Uno,0.000016,3447.175599,0.000032,3447.175646
4,Barangka,Fortune,0.000016,5929.364535,0.009198,5929.373749


In [47]:
# Convert to a Matrix
df_bus_travel_dist_mat = csv_bus_travel_dist_mat.drop(columns=["entry_cost","network_cost","exit_cost"])
df_bus_travel_dist_mat.head()

,origin_id,destination_id,total_cost
0,Barangka,Barangka,0.000000
1,Barangka,Calumpang,3216.299414
2,Barangka,Concepcion Dos,5861.487580
3,Barangka,Concepcion Uno,3447.175646
4,Barangka,Fortune,5929.373749


In [48]:
# assign centroid to a centroid_number
list_cen = df_bus_travel_dist_mat["origin_id"].unique()
df_cen_no = pd.DataFrame(list_cen, columns=["Centroid"])
cen_no = list(range(0,len(list_cen)))
df_cen_no["Number"] = cen_no
df_cen_no.set_index("Centroid", inplace=True)
df_cen_no.head()

,Number
Centroid,
Barangka,0
Calumpang,1
Concepcion Dos,2
Concepcion Uno,3
Fortune,4


In [49]:
def cen_number(centroid):
    number = df_cen_no["Number"][centroid]
    return number

In [50]:
df_bus_travel_dist_mat["origin_id"] = df_bus_travel_dist_mat["origin_id"].apply(cen_number)
df_bus_travel_dist_mat["destination_id"] = df_bus_travel_dist_mat["destination_id"].apply(cen_number)

In [51]:
df_bus_travel_dist_mat.head()

,origin_id,destination_id,total_cost
0,0,0,0.000000
1,0,1,3216.299414
2,0,2,5861.487580
3,0,3,3447.175646
4,0,4,5929.373749


In [52]:
rows = df_bus_travel_dist_mat["origin_id"].nunique()
cols = rows
 
dist_mat = [[0 for _ in range(cols)] for _ in range(rows)]

for a in range(0, len(df_bus_travel_dist_mat)):
    i = df_bus_travel_dist_mat["origin_id"][a]
    j = df_bus_travel_dist_mat["destination_id"][a]
    dist_mat[i][j] = df_bus_travel_dist_mat["total_cost"][a]

print(f'matrix of dimension {rows} x {cols} is {dist_mat}')

matrix of dimension 15 x 15 is [[0.0, 3216.2994136, 5861.4875803, 3447.1756463, 5929.3737488, 1036.4434799, 2896.572854, 4849.6456005, 6132.3279255, 5208.3365261, 2644.0540638, 2053.656446, 1713.7822483, 531.6994438, 4106.415622], [3216.2994136, 0.0, 6253.5320417, 3839.2201078, 6321.4182102, 2179.8569547, 3288.6173155, 5241.690062, 6524.372387, 5600.3809875, 886.1458906, 1486.4232269, 1816.4183854, 2684.6000048, 4498.4600835], [5861.4875803, 6253.5320417, 0.0, 2414.3119981, 4597.9158201, 4825.0451213, 2964.921968, 1011.8519305, 4800.8699968, 3876.8785974, 5681.2866919, 5090.8890741, 4751.0148764, 5329.7881715, 2774.9576934], [3447.1756463, 3839.2201078, 2414.3119981, 0.0, 2482.1981666, 2410.7331874, 550.6100341, 1402.4700183, 2685.1523433, 1761.1609439, 3266.9747579, 2676.5771402, 2336.7029425, 2915.4762375, 659.2400398], [5929.3737488, 6321.4182102, 4597.9158201, 2482.1981666, 0.0, 4892.9312899, 3032.8081365, 3586.0738404, 3725.5913484, 721.0372662, 5749.1728604, 5158.7752426, 4818.90

In [53]:
df_dist_mat = pd.DataFrame(dist_mat)
df_dist_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.000000,3216.299414,5861.487580,3447.175646,5929.373749,1036.443480,2896.572854,4849.645601,6132.327925,5208.336526,2644.054064,2053.656446,1713.782248,531.699444,4106.415622
1,3216.299414,0.000000,6253.532042,3839.220108,6321.418210,2179.856955,3288.617316,5241.690062,6524.372387,5600.380987,886.145891,1486.423227,1816.418385,2684.600005,4498.460084
2,5861.487580,6253.532042,0.000000,2414.311998,4597.915820,4825.045121,2964.921968,1011.851930,4800.869997,3876.878597,5681.286692,5090.889074,4751.014876,5329.788172,2774.957693
3,3447.175646,3839.220108,2414.311998,0.000000,2482.198167,2410.733187,550.610034,1402.470018,2685.152343,1761.160944,3266.974758,2676.577140,2336.702943,2915.476238,659.240040
4,5929.373749,6321.418210,4597.915820,2482.198167,0.000000,4892.931290,3032.808136,3586.073840,3725.591348,721.037266,5749.172860,5158.775243,4818.901045,5397.674340,1822.970290
5,1036.443480,2179.856955,4825.045121,2410.733187,4892.931290,0.000000,1860.130395,3813.203142,5095.885467,4171.894067,1607.611605,1017.213987,677.339789,504.744071,3069.973163
6,2896.572854,3288.617316,2964.921968,550.610034,3032.808136,1860.130395,0.000000,1953.079988,3235.762313,2311.770914,2716.371966,2125.974348,1786.100150,2364.873445,1209.850010
7,4849.645601,5241.690062,1011.851930,1402.470018,3586.073840,3813.203142,1953.079988,0.000000,3789.028017,2865.036618,4669.444712,4079.047094,3739.172897,4317.946192,1763.115714
8,6132.327925,6524.372387,4800.869997,2685.152343,3725.591348,5095.885467,3235.762313,3789.028017,0.000000,3004.554126,5952.127037,5361.729419,5021.855222,5600.628517,2025.924466
9,5208.336526,5600.380987,3876.878597,1761.160944,721.037266,4171.894067,2311.770914,2865.036618,3004.554126,0.000000,5028.135638,4437.738020,4097.863822,4676.637117,1101.933067


In [54]:
#for g in range(0, len(df_cost_mat)):
#    df_cost_mat.loc[g,g] = 300

#df_cost_mat

In [55]:
# Convert from total_cost (distance) to minute

def dist_to_min(dist):
    fixed_rate = 25    # kph
    dist = dist/1000   # convert to km
    min = (dist/fixed_rate)*60
    return min

df_bus_travel_time_mat = pd.DataFrame(df_dist_mat.apply(dist_to_min))
df_bus_travel_time_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.000000,7.719119,14.067570,8.273222,14.230497,2.487464,6.951775,11.639149,14.717587,12.500008,6.345730,4.928775,4.113077,1.276079,9.855397
1,7.719119,0.000000,15.008477,9.214128,15.171404,5.231657,7.892682,12.580056,15.658494,13.440914,2.126750,3.567416,4.359404,6.443040,10.796304
2,14.067570,15.008477,0.000000,5.794349,11.034998,11.580108,7.115813,2.428445,11.522088,9.304509,13.635088,12.218134,11.402436,12.791492,6.659898
3,8.273222,9.214128,5.794349,0.000000,5.957276,5.785760,1.321464,3.365928,6.444366,4.226786,7.840739,6.423785,5.608087,6.997143,1.582176
4,14.230497,15.171404,11.034998,5.957276,0.000000,11.743035,7.278740,8.606577,8.941419,1.730489,13.798015,12.381061,11.565363,12.954418,4.375129
5,2.487464,5.231657,11.580108,5.785760,11.743035,0.000000,4.464313,9.151688,12.230125,10.012546,3.858268,2.441314,1.625615,1.211386,7.367936
6,6.951775,7.892682,7.115813,1.321464,7.278740,4.464313,0.000000,4.687392,7.765830,5.548250,6.519293,5.102338,4.286640,5.675696,2.903640
7,11.639149,12.580056,2.428445,3.365928,8.606577,9.151688,4.687392,0.000000,9.093667,6.876088,11.206667,9.789713,8.974015,10.363071,4.231478
8,14.717587,15.658494,11.522088,6.444366,8.941419,12.230125,7.765830,9.093667,0.000000,7.210930,14.285105,12.868151,12.052453,13.441508,4.862219
9,12.500008,13.440914,9.304509,4.226786,1.730489,10.012546,5.548250,6.876088,7.210930,0.000000,12.067526,10.650571,9.834873,11.223929,2.644639


In [56]:
no_of_centroids = len(df_dist_mat.index)
r = no_of_centroids
c = no_of_centroids

first_five = 15  
every_km = 2    
     
final_cost_mat = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        dist = df_dist_mat[i][j]
        dist = dist/1000     # convert to km

        if (dist <= 5):
            rate = first_five
        else:
            rate = first_five + every_km * (dist - 5) 
        
        final_cost_mat[i][j] = round(rate,2)

print(f'matrix of dimension {r} x {c} is {final_cost_mat}')

matrix of dimension 15 x 15 is [[15, 15, 16.72, 15, 16.86, 15, 15, 15, 17.26, 15.42, 15, 15, 15, 15, 15], [15, 15, 17.51, 15, 17.64, 15, 15, 15.48, 18.05, 16.2, 15, 15, 15, 15, 15], [16.72, 17.51, 15, 15, 15, 15, 15, 15, 15, 15, 16.36, 15.18, 15, 15.66, 15], [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15], [16.86, 17.64, 15, 15, 15, 15, 15, 15, 15, 15, 16.5, 15.32, 15, 15.8, 15], [15, 15, 15, 15, 15, 15, 15, 15, 15.19, 15, 15, 15, 15, 15, 15], [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15], [15, 15.48, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15], [17.26, 18.05, 15, 15, 15, 15.19, 15, 15, 15, 15, 16.9, 15.72, 15.04, 16.2, 15], [15.42, 16.2, 15, 15, 15, 15, 15, 15, 15, 15, 15.06, 15, 15, 15, 15], [15, 15, 16.36, 15, 16.5, 15, 15, 15, 16.9, 15.06, 15, 15, 15, 15, 15], [15, 15, 15.18, 15, 15.32, 15, 15, 15, 15.72, 15, 15, 15, 15, 15, 15], [15, 15, 15, 15, 15, 15, 15, 15, 15.04, 15, 15, 15, 15, 15, 15], [15, 15, 15.66, 15, 15.8, 15, 15, 15, 16.2, 15, 15, 1

In [57]:
df_bus_travel_cost_mat = pd.DataFrame(final_cost_mat)
df_bus_travel_cost_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,15.00,15.00,16.72,15,16.86,15.00,15,15.00,17.26,15.42,15.00,15.00,15.00,15.00,15
1,15.00,15.00,17.51,15,17.64,15.00,15,15.48,18.05,16.20,15.00,15.00,15.00,15.00,15
2,16.72,17.51,15.00,15,15.00,15.00,15,15.00,15.00,15.00,16.36,15.18,15.00,15.66,15
3,15.00,15.00,15.00,15,15.00,15.00,15,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15
4,16.86,17.64,15.00,15,15.00,15.00,15,15.00,15.00,15.00,16.50,15.32,15.00,15.80,15
5,15.00,15.00,15.00,15,15.00,15.00,15,15.00,15.19,15.00,15.00,15.00,15.00,15.00,15
6,15.00,15.00,15.00,15,15.00,15.00,15,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15
7,15.00,15.48,15.00,15,15.00,15.00,15,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15
8,17.26,18.05,15.00,15,15.00,15.19,15,15.00,15.00,15.00,16.90,15.72,15.04,16.20,15
9,15.42,16.20,15.00,15,15.00,15.00,15,15.00,15.00,15.00,15.06,15.00,15.00,15.00,15


In [58]:
df_cost = df_cn_final["Centroids"].to_frame()
list_c = df_cost["Centroids"].to_list()

list_cn = []
for c in list_c:
    ccn = []
    for d in c:
        cn = cen_number(d)
        ccn.append(cn)
    list_cn.append(ccn)
        
df_cost["Centroid_Numbers"] = list_cn
df_cost

,Centroids,Centroid_Numbers
Node,,
Sta. Lucia Grandmall,[San Roque],[10]
Bayan Bayanan,"[Concepcion Dos, Concepcion Uno]","[2, 3]"
Ligaya,[Calumpang],[1]
Riverbanks,[Barangka],[0]
Marikina Bayan,"[Jesus de La Pena, Santo Nino, Santa Elena]","[5, 12, 11]"
Tumana,"[Tumana, Marikina Heights]","[14, 7]"
Nangka,"[Fortune, Nangka]","[4, 8]"
Fairlane,[Parang],[9]
Golden Valley,[Malanday],[6]


In [59]:
# TENTATIVE (for sample only)
list_node_numbers = range(0,len(df_cost.index))
list_node_numbers

range(0, 10)

In [60]:
df_cost["Node_Number"] = list_node_numbers
df_cost

,Centroids,Centroid_Numbers,Node_Number
Node,,,
Sta. Lucia Grandmall,[San Roque],[10],0
Bayan Bayanan,"[Concepcion Dos, Concepcion Uno]","[2, 3]",1
Ligaya,[Calumpang],[1],2
Riverbanks,[Barangka],[0],3
Marikina Bayan,"[Jesus de La Pena, Santo Nino, Santa Elena]","[5, 12, 11]",4
Tumana,"[Tumana, Marikina Heights]","[14, 7]",5
Nangka,"[Fortune, Nangka]","[4, 8]",6
Fairlane,[Parang],[9],7
Golden Valley,[Malanday],[6],8


In [61]:
df_nn_cn = df_cost[["Node_Number","Centroid_Numbers"]].set_index("Node_Number")
df_nn_cn

,Centroid_Numbers
Node_Number,
0,[10]
1,"[2, 3]"
2,[1]
3,[0]
4,"[5, 12, 11]"
5,"[14, 7]"
6,"[4, 8]"
7,[9]
8,[6]


In [62]:
no_of_nodes = len(df_nn_cn.index)
no_of_nodes

10

In [63]:
df_bus_travel_time_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.000000,7.719119,14.067570,8.273222,14.230497,2.487464,6.951775,11.639149,14.717587,12.500008,6.345730,4.928775,4.113077,1.276079,9.855397
1,7.719119,0.000000,15.008477,9.214128,15.171404,5.231657,7.892682,12.580056,15.658494,13.440914,2.126750,3.567416,4.359404,6.443040,10.796304
2,14.067570,15.008477,0.000000,5.794349,11.034998,11.580108,7.115813,2.428445,11.522088,9.304509,13.635088,12.218134,11.402436,12.791492,6.659898
3,8.273222,9.214128,5.794349,0.000000,5.957276,5.785760,1.321464,3.365928,6.444366,4.226786,7.840739,6.423785,5.608087,6.997143,1.582176
4,14.230497,15.171404,11.034998,5.957276,0.000000,11.743035,7.278740,8.606577,8.941419,1.730489,13.798015,12.381061,11.565363,12.954418,4.375129
5,2.487464,5.231657,11.580108,5.785760,11.743035,0.000000,4.464313,9.151688,12.230125,10.012546,3.858268,2.441314,1.625615,1.211386,7.367936
6,6.951775,7.892682,7.115813,1.321464,7.278740,4.464313,0.000000,4.687392,7.765830,5.548250,6.519293,5.102338,4.286640,5.675696,2.903640
7,11.639149,12.580056,2.428445,3.365928,8.606577,9.151688,4.687392,0.000000,9.093667,6.876088,11.206667,9.789713,8.974015,10.363071,4.231478
8,14.717587,15.658494,11.522088,6.444366,8.941419,12.230125,7.765830,9.093667,0.000000,7.210930,14.285105,12.868151,12.052453,13.441508,4.862219
9,12.500008,13.440914,9.304509,4.226786,1.730489,10.012546,5.548250,6.876088,7.210930,0.000000,12.067526,10.650571,9.834873,11.223929,2.644639


In [64]:
r = no_of_nodes
c = no_of_nodes
 
final_node_time_mat = [[0 for _ in range(c)] for _ in range(r)]
final_node_cost_mat = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        list_i = df_nn_cn["Centroid_Numbers"][i]
        list_j = df_nn_cn["Centroid_Numbers"][j]

        sum_cost = 0
        sum_time = 0
        counter = 0
        for a in list_i:
            for b in list_j:
                sum_time = sum_time + df_bus_travel_time_mat[a][b]
                sum_cost = sum_cost + df_bus_travel_cost_mat[a][b]
                counter = counter + 1
        
        final_node_time_mat[i][j] = round(sum_time/counter,2)
        final_node_cost_mat[i][j] = round(sum_cost/counter,2)

print(f'travel time matrix of dimension {r} x {c} is {final_node_time_mat}')
print(f'travel cost matrix of dimension {r} x {c} is {final_node_cost_mat}')

travel time matrix of dimension 10 x 10 is [[0.0, 10.74, 2.13, 6.35, 2.51, 10.31, 14.04, 12.07, 6.52, 5.07], [10.74, 2.9, 12.11, 11.17, 8.84, 3.51, 8.74, 6.77, 4.22, 9.89], [2.13, 12.11, 0.0, 7.72, 4.39, 11.69, 15.41, 13.44, 7.89, 6.44], [6.35, 11.17, 7.72, 0.0, 3.84, 10.75, 14.47, 12.5, 6.95, 1.28], [2.51, 8.84, 4.39, 3.84, 1.09, 8.41, 12.14, 10.17, 4.62, 2.57], [10.31, 3.51, 11.69, 10.75, 8.41, 2.12, 6.73, 4.76, 3.8, 9.47], [14.04, 8.74, 15.41, 14.47, 12.14, 6.73, 4.47, 4.47, 7.52, 13.2], [12.07, 6.77, 13.44, 12.5, 10.17, 4.76, 4.47, 0.0, 5.55, 11.22], [6.52, 4.22, 7.89, 6.95, 4.62, 3.8, 7.52, 5.55, 0.0, 5.68], [5.07, 9.89, 6.44, 1.28, 2.57, 9.47, 13.2, 11.22, 5.68, 0.0]]
travel cost matrix of dimension 10 x 10 is [[15.0, 15.68, 15.0, 15.0, 15.0, 15.0, 16.7, 15.06, 15.0, 15.0], [15.68, 15.0, 16.26, 15.86, 15.03, 15.0, 15.0, 15.0, 15.0, 15.33], [15.0, 16.26, 15.0, 15.0, 15.0, 15.24, 17.84, 16.2, 15.0, 15.0], [15.0, 15.86, 15.0, 15.0, 15.0, 15.0, 17.06, 15.42, 15.0, 15.0], [15.0, 15.03

In [65]:
bus_travel_time_mat = pd.DataFrame(final_node_time_mat)
bus_travel_time_mat

,0,1,2,3,4,5,6,7,8,9
0,0.00,10.74,2.13,6.35,2.51,10.31,14.04,12.07,6.52,5.07
1,10.74,2.90,12.11,11.17,8.84,3.51,8.74,6.77,4.22,9.89
2,2.13,12.11,0.00,7.72,4.39,11.69,15.41,13.44,7.89,6.44
3,6.35,11.17,7.72,0.00,3.84,10.75,14.47,12.50,6.95,1.28
4,2.51,8.84,4.39,3.84,1.09,8.41,12.14,10.17,4.62,2.57
5,10.31,3.51,11.69,10.75,8.41,2.12,6.73,4.76,3.80,9.47
6,14.04,8.74,15.41,14.47,12.14,6.73,4.47,4.47,7.52,13.20
7,12.07,6.77,13.44,12.50,10.17,4.76,4.47,0.00,5.55,11.22
8,6.52,4.22,7.89,6.95,4.62,3.80,7.52,5.55,0.00,5.68
9,5.07,9.89,6.44,1.28,2.57,9.47,13.20,11.22,5.68,0.00


In [66]:
bus_travel_cost_mat = pd.DataFrame(final_node_cost_mat)
bus_travel_cost_mat

,0,1,2,3,4,5,6,7,8,9
0,15.00,15.68,15.00,15.00,15.00,15.00,16.70,15.06,15.0,15.00
1,15.68,15.00,16.26,15.86,15.03,15.00,15.00,15.00,15.0,15.33
2,15.00,16.26,15.00,15.00,15.00,15.24,17.84,16.20,15.0,15.00
3,15.00,15.86,15.00,15.00,15.00,15.00,17.06,15.42,15.0,15.00
4,15.00,15.03,15.00,15.00,15.00,15.00,15.21,15.00,15.0,15.00
5,15.00,15.00,15.24,15.00,15.00,15.00,15.00,15.00,15.0,15.00
6,16.70,15.00,17.84,17.06,15.21,15.00,15.00,15.00,15.0,16.00
7,15.06,15.00,16.20,15.42,15.00,15.00,15.00,15.00,15.0,15.00
8,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.0,15.00
9,15.00,15.33,15.00,15.00,15.00,15.00,16.00,15.00,15.0,15.00


#### 9.1.b. Car

'highway'='primary' OR 'highway'='secondary' OR 'highway'='tertiary' OR 'highway'='motorway' OR 'route'='road'

In [67]:
csv_car_travel_dist_mat = pd.read_csv("car_cost_matrix.csv")
#csv_car_travel_dist_mat.head()

# Convert to a Matrix
df_car_travel_dist_mat = csv_car_travel_dist_mat.drop(columns=["entry_cost","network_cost","exit_cost"])
df_car_travel_dist_mat.head()

,origin_id,destination_id,total_cost
0,Barangka,Barangka,0.000000
1,Barangka,Calumpang,2173.734483
2,Barangka,Concepcion Dos,4446.194963
3,Barangka,Concepcion Uno,3723.921161
4,Barangka,Fortune,5552.725543


In [68]:
# assign centroid to a centroid_number
list_cen = df_car_travel_dist_mat["origin_id"].unique()
df_cen_no = pd.DataFrame(list_cen, columns=["Centroid"])
cen_no = list(range(0,len(list_cen)))
df_cen_no["Number"] = cen_no
df_cen_no.set_index("Centroid", inplace=True)
df_cen_no.head()

,Number
Centroid,
Barangka,0
Calumpang,1
Concepcion Dos,2
Concepcion Uno,3
Fortune,4


In [69]:
def cen_number(centroid):
    number = df_cen_no["Number"][centroid]
    return number

df_car_travel_dist_mat["origin_id"] = df_car_travel_dist_mat["origin_id"].apply(cen_number)
df_car_travel_dist_mat["destination_id"] = df_car_travel_dist_mat["destination_id"].apply(cen_number)

df_car_travel_dist_mat.head()

,origin_id,destination_id,total_cost
0,0,0,0.000000
1,0,1,2173.734483
2,0,2,4446.194963
3,0,3,3723.921161
4,0,4,5552.725543


In [70]:
rows = df_car_travel_dist_mat["origin_id"].nunique()
cols = rows
 
dist_mat = [[0 for _ in range(cols)] for _ in range(rows)]

for a in range(0, len(df_car_travel_dist_mat)):
    i = df_car_travel_dist_mat["origin_id"][a]
    j = df_car_travel_dist_mat["destination_id"][a]
    dist_mat[i][j] = df_car_travel_dist_mat["total_cost"][a]

print(f'matrix of dimension {rows} x {cols} is {dist_mat}')

matrix of dimension 15 x 15 is [[0.0, 2173.7344833, 4446.194963, 3723.9211608, 5552.7255428, 207.1681625, 208.1919553, 5307.0329915, 4969.68416, 4999.814287, 2866.9403533, 2895.9253149, 2969.4765114, 128.1579457, 240.7725812], [2173.7344833, 0.0, 2272.470009, 1550.1962068, 3379.0005888, 1966.5927884, 2381.9191453, 3133.3080375, 2795.959206, 2826.089333, 693.2153993, 722.2003609, 795.7515574, 2045.5933374, 2414.4997712], [4446.194963, 2272.470009, 0.0, 722.3299336, 1106.5611701, 4239.0532681, 4654.379625, 860.8686188, 523.5197873, 553.6499143, 1579.2702384, 1550.2946997, 1476.7512436, 4318.0538171, 4686.9602509], [3723.9211608, 1550.1962068, 722.3299336, 0.0, 1828.8605134, 3516.7794659, 3932.1058227, 1583.1679621, 1245.8191306, 1275.9492576, 856.9964361, 828.0208974, 754.4774414, 3595.7800149, 3964.6864486], [5552.7255428, 3379.0005888, 1106.5611701, 1828.8605134, 0.0, 5345.5838479, 5760.9102047, 245.7433773, 583.1391319, 552.9812918, 2685.8008181, 2656.8252794, 2583.2818234, 5424.58439

In [71]:
df_dist_mat = pd.DataFrame(dist_mat)
df_dist_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.000000,2173.734483,4446.194963,3723.921161,5552.725543,207.168162,208.191955,5307.032992,4969.684160,4999.814287,2866.940353,2895.925315,2969.476511,128.157946,240.772581
1,2173.734483,0.000000,2272.470009,1550.196207,3379.000589,1966.592788,2381.919145,3133.308037,2795.959206,2826.089333,693.215399,722.200361,795.751557,2045.593337,2414.499771
2,4446.194963,2272.470009,0.000000,722.329934,1106.561170,4239.053268,4654.379625,860.868619,523.519787,553.649914,1579.270238,1550.294700,1476.751244,4318.053817,4686.960251
3,3723.921161,1550.196207,722.329934,0.000000,1828.860513,3516.779466,3932.105823,1583.167962,1245.819131,1275.949258,856.996436,828.020897,754.477441,3595.780015,3964.686449
4,5552.725543,3379.000589,1106.561170,1828.860513,0.000000,5345.583848,5760.910205,245.743377,583.139132,552.981292,2685.800818,2656.825279,2583.281823,5424.584397,5793.490831
5,207.168162,1966.592788,4239.053268,3516.779466,5345.583848,0.000000,415.352824,5099.891297,4762.542465,4792.672592,2659.798658,2688.783620,2762.334817,79.027017,447.933450
6,208.191955,2381.919145,4654.379625,3932.105823,5760.910205,415.352824,0.000000,5515.217653,5177.868822,5207.998949,3075.125015,3104.109977,3177.661173,336.342608,32.628712
7,5307.032992,3133.308037,860.868619,1583.167962,245.743377,5099.891297,5515.217653,0.000000,337.446581,307.288741,2440.108267,2411.132728,2337.589272,5178.891846,5547.798279
8,4969.684160,2795.959206,523.519787,1245.819131,583.139132,4762.542465,5177.868822,337.446581,0.000000,30.227876,2102.759435,2073.783897,2000.240441,4841.543014,5210.449448
9,4999.814287,2826.089333,553.649914,1275.949258,552.981292,4792.672592,5207.998949,307.288741,30.227876,0.000000,2132.889562,2103.914024,2030.370568,4871.673141,5240.579575


In [72]:
# Convert from total_cost (distance) to minute

def dist_to_min(dist):
    fixed_rate = 25    # kph
    dist = dist/1000   # convert to km
    min = (dist/fixed_rate)*60
    return min

df_car_travel_time_mat = pd.DataFrame(df_dist_mat.apply(dist_to_min))
df_car_travel_time_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.000000,5.216963,10.670868,8.937411,13.326541,0.497204,0.499661,12.736879,11.927242,11.999554,6.880657,6.950221,7.126744,0.307579,0.577854
1,5.216963,0.000000,5.453928,3.720471,8.109601,4.719823,5.716606,7.519939,6.710302,6.782614,1.663717,1.733281,1.909804,4.909424,5.794799
2,10.670868,5.453928,0.000000,1.733592,2.655747,10.173728,11.170511,2.066085,1.256447,1.328760,3.790249,3.720707,3.544203,10.363329,11.248705
3,8.937411,3.720471,1.733592,0.000000,4.389265,8.440271,9.437054,3.799603,2.989966,3.062278,2.056791,1.987250,1.810746,8.629872,9.515247
4,13.326541,8.109601,2.655747,4.389265,0.000000,12.829401,13.826184,0.589784,1.399534,1.327155,6.445922,6.376381,6.199876,13.019003,13.904378
5,0.497204,4.719823,10.173728,8.440271,12.829401,0.000000,0.996847,12.239739,11.430102,11.502414,6.383517,6.453081,6.629604,0.189665,1.075040
6,0.499661,5.716606,11.170511,9.437054,13.826184,0.996847,0.000000,13.236522,12.426885,12.499197,7.380300,7.449864,7.626387,0.807222,0.078309
7,12.736879,7.519939,2.066085,3.799603,0.589784,12.239739,13.236522,0.000000,0.809872,0.737493,5.856260,5.786719,5.610214,12.429340,13.314716
8,11.927242,6.710302,1.256447,2.989966,1.399534,11.430102,12.426885,0.809872,0.000000,0.072547,5.046623,4.977081,4.800577,11.619703,12.505079
9,11.999554,6.782614,1.328760,3.062278,1.327155,11.502414,12.499197,0.737493,0.072547,0.000000,5.118935,5.049394,4.872889,11.692016,12.577391


In [73]:
no_of_centroids = len(df_dist_mat.index)
r = no_of_centroids
c = no_of_centroids

base_fare = 40  
every_km = 15    
     
final_cost_mat = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        dist = df_dist_mat[i][j]
        dist = dist/1000     # convert to km

        rate = base_fare + every_km * dist
        
        final_cost_mat[i][j] = round(rate,2)

print(f'matrix of dimension {r} x {c} is {final_cost_mat}')

matrix of dimension 15 x 15 is [[40.0, 72.61, 106.69, 95.86, 123.29, 43.11, 43.12, 119.61, 114.55, 115.0, 83.0, 83.44, 84.54, 41.92, 43.61], [72.61, 40.0, 74.09, 63.25, 90.69, 69.5, 75.73, 87.0, 81.94, 82.39, 50.4, 50.83, 51.94, 70.68, 76.22], [106.69, 74.09, 40.0, 50.83, 56.6, 103.59, 109.82, 52.91, 47.85, 48.3, 63.69, 63.25, 62.15, 104.77, 110.3], [95.86, 63.25, 50.83, 40.0, 67.43, 92.75, 98.98, 63.75, 58.69, 59.14, 52.85, 52.42, 51.32, 93.94, 99.47], [123.29, 90.69, 56.6, 67.43, 40.0, 120.18, 126.41, 43.69, 48.75, 48.29, 80.29, 79.85, 78.75, 121.37, 126.9], [43.11, 69.5, 103.59, 92.75, 120.18, 40.0, 46.23, 116.5, 111.44, 111.89, 79.9, 80.33, 81.44, 41.19, 46.72], [43.12, 75.73, 109.82, 98.98, 126.41, 46.23, 40.0, 122.73, 117.67, 118.12, 86.13, 86.56, 87.66, 45.05, 40.49], [119.61, 87.0, 52.91, 63.75, 43.69, 116.5, 122.73, 40.0, 45.06, 44.61, 76.6, 76.17, 75.06, 117.68, 123.22], [114.55, 81.94, 47.85, 58.69, 48.75, 111.44, 117.67, 45.06, 40.0, 40.45, 71.54, 71.11, 70.0, 112.62, 118.1

In [74]:
df_car_travel_cost_mat = pd.DataFrame(final_cost_mat)
df_car_travel_cost_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,40.00,72.61,106.69,95.86,123.29,43.11,43.12,119.61,114.55,115.00,83.00,83.44,84.54,41.92,43.61
1,72.61,40.00,74.09,63.25,90.69,69.50,75.73,87.00,81.94,82.39,50.40,50.83,51.94,70.68,76.22
2,106.69,74.09,40.00,50.83,56.60,103.59,109.82,52.91,47.85,48.30,63.69,63.25,62.15,104.77,110.30
3,95.86,63.25,50.83,40.00,67.43,92.75,98.98,63.75,58.69,59.14,52.85,52.42,51.32,93.94,99.47
4,123.29,90.69,56.60,67.43,40.00,120.18,126.41,43.69,48.75,48.29,80.29,79.85,78.75,121.37,126.90
5,43.11,69.50,103.59,92.75,120.18,40.00,46.23,116.50,111.44,111.89,79.90,80.33,81.44,41.19,46.72
6,43.12,75.73,109.82,98.98,126.41,46.23,40.00,122.73,117.67,118.12,86.13,86.56,87.66,45.05,40.49
7,119.61,87.00,52.91,63.75,43.69,116.50,122.73,40.00,45.06,44.61,76.60,76.17,75.06,117.68,123.22
8,114.55,81.94,47.85,58.69,48.75,111.44,117.67,45.06,40.00,40.45,71.54,71.11,70.00,112.62,118.16
9,115.00,82.39,48.30,59.14,48.29,111.89,118.12,44.61,40.45,40.00,71.99,71.56,70.46,113.08,118.61


In [75]:
r = no_of_nodes
c = no_of_nodes
 
final_node_time_mat = [[0 for _ in range(c)] for _ in range(r)]
final_node_cost_mat = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        list_i = df_nn_cn["Centroid_Numbers"][i]
        list_j = df_nn_cn["Centroid_Numbers"][j]

        sum_cost = 0
        sum_time = 0
        counter = 0
        for a in list_i:
            for b in list_j:
                sum_time = sum_time + df_car_travel_time_mat[a][b]
                sum_cost = sum_cost + df_car_travel_cost_mat[a][b]
                counter = counter + 1
        
        final_node_time_mat[i][j] = round(sum_time/counter,2)
        final_node_cost_mat[i][j] = round(sum_cost/counter,2)

print(f'travel time matrix of dimension {r} x {c} is {final_node_time_mat}')
print(f'travel cost matrix of dimension {r} x {c} is {final_node_cost_mat}')

travel time matrix of dimension 10 x 10 is [[0.0, 2.92, 1.66, 6.88, 2.23, 6.66, 5.75, 5.12, 7.38, 6.57], [2.92, 0.87, 4.59, 9.8, 4.95, 6.66, 2.82, 2.2, 10.3, 9.5], [1.66, 4.59, 0.0, 5.22, 2.79, 6.66, 7.41, 6.78, 5.72, 4.91], [6.88, 9.8, 5.22, 0.0, 4.86, 6.66, 12.63, 12.0, 0.5, 0.31], [2.23, 4.95, 2.79, 4.86, 2.95, 6.66, 7.77, 7.14, 5.36, 4.55], [6.66, 6.66, 6.66, 6.66, 6.66, 6.66, 6.95, 6.66, 6.66, 6.66], [5.75, 2.82, 7.41, 12.63, 7.77, 6.95, 0.7, 0.7, 13.13, 12.32], [5.12, 2.2, 6.78, 12.0, 7.14, 6.66, 0.7, 0.0, 12.5, 11.69], [7.38, 10.3, 5.72, 0.5, 5.36, 6.66, 13.13, 12.5, 0.0, 0.81], [6.57, 9.5, 4.91, 0.31, 4.55, 6.66, 12.32, 11.69, 0.81, 0.0]]
travel cost matrix of dimension 10 x 10 is [[40.0, 58.27, 50.4, 83.0, 53.96, 81.61, 75.92, 71.99, 86.13, 81.08], [58.27, 45.42, 68.67, 101.28, 70.91, 81.61, 57.64, 53.72, 104.4, 99.35], [50.4, 68.67, 40.0, 72.61, 57.42, 81.61, 86.32, 82.39, 75.73, 70.68], [83.0, 101.28, 72.61, 40.0, 70.36, 81.61, 118.92, 115.0, 43.12, 41.92], [53.96, 70.91, 57

In [76]:
car_travel_time_mat = pd.DataFrame(final_node_time_mat)
car_travel_time_mat

,0,1,2,3,4,5,6,7,8,9
0,0.00,2.92,1.66,6.88,2.23,6.66,5.75,5.12,7.38,6.57
1,2.92,0.87,4.59,9.80,4.95,6.66,2.82,2.20,10.30,9.50
2,1.66,4.59,0.00,5.22,2.79,6.66,7.41,6.78,5.72,4.91
3,6.88,9.80,5.22,0.00,4.86,6.66,12.63,12.00,0.50,0.31
4,2.23,4.95,2.79,4.86,2.95,6.66,7.77,7.14,5.36,4.55
5,6.66,6.66,6.66,6.66,6.66,6.66,6.95,6.66,6.66,6.66
6,5.75,2.82,7.41,12.63,7.77,6.95,0.70,0.70,13.13,12.32
7,5.12,2.20,6.78,12.00,7.14,6.66,0.70,0.00,12.50,11.69
8,7.38,10.30,5.72,0.50,5.36,6.66,13.13,12.50,0.00,0.81
9,6.57,9.50,4.91,0.31,4.55,6.66,12.32,11.69,0.81,0.00


In [77]:
car_travel_cost_mat = pd.DataFrame(final_node_cost_mat)
car_travel_cost_mat

,0,1,2,3,4,5,6,7,8,9
0,40.00,58.27,50.40,83.00,53.96,81.61,75.92,71.99,86.13,81.08
1,58.27,45.42,68.67,101.28,70.91,81.61,57.64,53.72,104.40,99.35
2,50.40,68.67,40.00,72.61,57.42,81.61,86.32,82.39,75.73,70.68
3,83.00,101.28,72.61,40.00,70.36,81.61,118.92,115.00,43.12,41.92
4,53.96,70.91,57.42,70.36,58.42,81.61,88.56,84.64,73.48,68.44
5,81.61,81.61,81.61,81.61,81.61,81.61,83.45,81.61,81.61,81.60
6,75.92,57.64,86.32,118.92,88.56,83.45,44.38,44.37,122.04,117.00
7,71.99,53.72,82.39,115.00,84.64,81.61,44.37,40.00,118.12,113.08
8,86.13,104.40,75.73,43.12,73.48,81.61,122.04,118.12,40.00,45.05
9,81.08,99.35,70.68,41.92,68.44,81.60,117.00,113.08,45.05,40.00


#### 9.1.c. Motorcycle

'highway'='primary' OR 'highway'='secondary' OR 'highway'='tertiary' OR 'highway'='motorway' OR 'route'='road'

OR 'highway'='primary_link' OR 'highway'='secondary_link' OR 'highway'='tertiary_link' OR 'highway'='motorway_link'

In [78]:
csv_motor_travel_dist_mat = pd.read_csv("motor_cost_matrix.csv")
#csv_motor_travel_dist_mat.head()

# Convert to a Matrix
df_motor_travel_dist_mat = csv_motor_travel_dist_mat.drop(columns=["entry_cost","network_cost","exit_cost"])
df_motor_travel_dist_mat.head()

,origin_id,destination_id,total_cost
0,Barangka,Barangka,0.000000
1,Barangka,Calumpang,2173.734483
2,Barangka,Concepcion Dos,4446.194963
3,Barangka,Concepcion Uno,3723.921161
4,Barangka,Fortune,5552.725543


In [79]:
# assign centroid to a centroid_number
list_cen = df_motor_travel_dist_mat["origin_id"].unique()
df_cen_no = pd.DataFrame(list_cen, columns=["Centroid"])
cen_no = list(range(0,len(list_cen)))
df_cen_no["Number"] = cen_no
df_cen_no.set_index("Centroid", inplace=True)
df_cen_no.head()

,Number
Centroid,
Barangka,0
Calumpang,1
Concepcion Dos,2
Concepcion Uno,3
Fortune,4


In [80]:
def cen_number(centroid):
    number = df_cen_no["Number"][centroid]
    return number

df_motor_travel_dist_mat["origin_id"] = df_motor_travel_dist_mat["origin_id"].apply(cen_number)
df_motor_travel_dist_mat["destination_id"] = df_motor_travel_dist_mat["destination_id"].apply(cen_number)

df_motor_travel_dist_mat.head()

,origin_id,destination_id,total_cost
0,0,0,0.000000
1,0,1,2173.734483
2,0,2,4446.194963
3,0,3,3723.921161
4,0,4,5552.725543


In [81]:
rows = df_motor_travel_dist_mat["origin_id"].nunique()
cols = rows
 
dist_mat = [[0 for _ in range(cols)] for _ in range(rows)]

for a in range(0, len(df_motor_travel_dist_mat)):
    i = df_motor_travel_dist_mat["origin_id"][a]
    j = df_motor_travel_dist_mat["destination_id"][a]
    dist_mat[i][j] = df_motor_travel_dist_mat["total_cost"][a]

print(f'matrix of dimension {rows} x {cols} is {dist_mat}')

matrix of dimension 15 x 15 is [[0.0, 2173.7344833, 4446.194963, 3723.9211608, 5552.7255428, 207.1681625, 208.1919553, 5307.0329915, 4969.68416, 4999.814287, 2866.9403533, 2895.9253149, 2969.4765114, 128.1579457, 240.7725812], [2173.7344833, 0.0, 2272.470009, 1550.1962068, 3379.0005888, 1966.5927884, 2381.9191453, 3133.3080375, 2795.959206, 2826.089333, 693.2153993, 722.2003609, 795.7515574, 2045.5933374, 2414.4997712], [4446.194963, 2272.470009, 0.0, 722.3299336, 1106.5611701, 4239.0532681, 4654.379625, 860.8686188, 523.5197873, 553.6499143, 1579.2702384, 1550.2946997, 1476.7512436, 4318.0538171, 4686.9602509], [3723.9211608, 1550.1962068, 722.3299336, 0.0, 1828.8605134, 3516.7794659, 3932.1058227, 1583.1679621, 1245.8191306, 1275.9492576, 856.9964361, 828.0208974, 754.4774414, 3595.7800149, 3964.6864486], [5552.7255428, 3379.0005888, 1106.5611701, 1828.8605134, 0.0, 5345.5838479, 5760.9102047, 245.7433773, 583.1391319, 552.9812918, 2685.8008181, 2656.8252794, 2583.2818234, 5424.58439

In [82]:
df_dist_mat = pd.DataFrame(dist_mat)
df_dist_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.000000,2173.734483,4446.194963,3723.921161,5552.725543,207.168162,208.191955,5307.032992,4969.684160,4999.814287,2866.940353,2895.925315,2969.476511,128.157946,240.772581
1,2173.734483,0.000000,2272.470009,1550.196207,3379.000589,1966.592788,2381.919145,3133.308037,2795.959206,2826.089333,693.215399,722.200361,795.751557,2045.593337,2414.499771
2,4446.194963,2272.470009,0.000000,722.329934,1106.561170,4239.053268,4654.379625,860.868619,523.519787,553.649914,1579.270238,1550.294700,1476.751244,4318.053817,4686.960251
3,3723.921161,1550.196207,722.329934,0.000000,1828.860513,3516.779466,3932.105823,1583.167962,1245.819131,1275.949258,856.996436,828.020897,754.477441,3595.780015,3964.686449
4,5552.725543,3379.000589,1106.561170,1828.860513,0.000000,5345.583848,5760.910205,245.743377,583.139132,552.981292,2685.800818,2656.825279,2583.281823,5424.584397,5793.490831
5,207.168162,1966.592788,4239.053268,3516.779466,5345.583848,0.000000,415.352824,5099.891297,4762.542465,4792.672592,2659.798658,2688.783620,2762.334817,79.027017,447.933450
6,208.191955,2381.919145,4654.379625,3932.105823,5760.910205,415.352824,0.000000,5515.217653,5177.868822,5207.998949,3075.125015,3104.109977,3177.661173,336.342608,32.628712
7,5307.032992,3133.308037,860.868619,1583.167962,245.743377,5099.891297,5515.217653,0.000000,337.446581,307.288741,2440.108267,2411.132728,2337.589272,5178.891846,5547.798279
8,4969.684160,2795.959206,523.519787,1245.819131,583.139132,4762.542465,5177.868822,337.446581,0.000000,30.227876,2102.759435,2073.783897,2000.240441,4841.543014,5210.449448
9,4999.814287,2826.089333,553.649914,1275.949258,552.981292,4792.672592,5207.998949,307.288741,30.227876,0.000000,2132.889562,2103.914024,2030.370568,4871.673141,5240.579575


In [83]:
# Convert from total_cost (distance) to minute

def dist_to_min(dist):
    fixed_rate = 25    # kph
    dist = dist/1000   # convert to km
    min = (dist/fixed_rate)*60
    return min

df_motor_travel_time_mat = pd.DataFrame(df_dist_mat.apply(dist_to_min))
df_motor_travel_time_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.000000,5.216963,10.670868,8.937411,13.326541,0.497204,0.499661,12.736879,11.927242,11.999554,6.880657,6.950221,7.126744,0.307579,0.577854
1,5.216963,0.000000,5.453928,3.720471,8.109601,4.719823,5.716606,7.519939,6.710302,6.782614,1.663717,1.733281,1.909804,4.909424,5.794799
2,10.670868,5.453928,0.000000,1.733592,2.655747,10.173728,11.170511,2.066085,1.256447,1.328760,3.790249,3.720707,3.544203,10.363329,11.248705
3,8.937411,3.720471,1.733592,0.000000,4.389265,8.440271,9.437054,3.799603,2.989966,3.062278,2.056791,1.987250,1.810746,8.629872,9.515247
4,13.326541,8.109601,2.655747,4.389265,0.000000,12.829401,13.826184,0.589784,1.399534,1.327155,6.445922,6.376381,6.199876,13.019003,13.904378
5,0.497204,4.719823,10.173728,8.440271,12.829401,0.000000,0.996847,12.239739,11.430102,11.502414,6.383517,6.453081,6.629604,0.189665,1.075040
6,0.499661,5.716606,11.170511,9.437054,13.826184,0.996847,0.000000,13.236522,12.426885,12.499197,7.380300,7.449864,7.626387,0.807222,0.078309
7,12.736879,7.519939,2.066085,3.799603,0.589784,12.239739,13.236522,0.000000,0.809872,0.737493,5.856260,5.786719,5.610214,12.429340,13.314716
8,11.927242,6.710302,1.256447,2.989966,1.399534,11.430102,12.426885,0.809872,0.000000,0.072547,5.046623,4.977081,4.800577,11.619703,12.505079
9,11.999554,6.782614,1.328760,3.062278,1.327155,11.502414,12.499197,0.737493,0.072547,0.000000,5.118935,5.049394,4.872889,11.692016,12.577391


In [84]:
no_of_centroids = len(df_dist_mat.index)
r = no_of_centroids
c = no_of_centroids

first_two = 50
every_km = 10    
     
final_cost_mat = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        dist = df_dist_mat[i][j]
        dist = dist/1000     # convert to km

        if (dist <= 2):
            rate = first_two
        else:
            rate = first_two + every_km * (dist - 2) 
        
        final_cost_mat[i][j] = round(rate,2)

print(f'matrix of dimension {r} x {c} is {final_cost_mat}')

matrix of dimension 15 x 15 is [[50, 51.74, 74.46, 67.24, 85.53, 50, 50, 83.07, 79.7, 80.0, 58.67, 58.96, 59.69, 50, 50], [51.74, 50, 52.72, 50, 63.79, 50, 53.82, 61.33, 57.96, 58.26, 50, 50, 50, 50.46, 54.14], [74.46, 52.72, 50, 50, 50, 72.39, 76.54, 50, 50, 50, 50, 50, 50, 73.18, 76.87], [67.24, 50, 50, 50, 50, 65.17, 69.32, 50, 50, 50, 50, 50, 50, 65.96, 69.65], [85.53, 63.79, 50, 50, 50, 83.46, 87.61, 50, 50, 50, 56.86, 56.57, 55.83, 84.25, 87.93], [50, 50, 72.39, 65.17, 83.46, 50, 50, 81.0, 77.63, 77.93, 56.6, 56.89, 57.62, 50, 50], [50, 53.82, 76.54, 69.32, 87.61, 50, 50, 85.15, 81.78, 82.08, 60.75, 61.04, 61.78, 50, 50], [83.07, 61.33, 50, 50, 50, 81.0, 85.15, 50, 50, 50, 54.4, 54.11, 53.38, 81.79, 85.48], [79.7, 57.96, 50, 50, 50, 77.63, 81.78, 50, 50, 50, 51.03, 50.74, 50.0, 78.42, 82.1], [80.0, 58.26, 50, 50, 50, 77.93, 82.08, 50, 50, 50, 51.33, 51.04, 50.3, 78.72, 82.41], [58.67, 50, 50, 50, 56.86, 56.6, 60.75, 54.4, 51.03, 51.33, 50, 50, 50, 57.39, 61.08], [58.96, 50, 50, 5

In [85]:
df_motor_travel_cost_mat = pd.DataFrame(final_cost_mat)
df_motor_travel_cost_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,50.00,51.74,74.46,67.24,85.53,50.00,50.00,83.07,79.70,80.00,58.67,58.96,59.69,50.00,50.00
1,51.74,50.00,52.72,50.00,63.79,50.00,53.82,61.33,57.96,58.26,50.00,50.00,50.00,50.46,54.14
2,74.46,52.72,50.00,50.00,50.00,72.39,76.54,50.00,50.00,50.00,50.00,50.00,50.00,73.18,76.87
3,67.24,50.00,50.00,50.00,50.00,65.17,69.32,50.00,50.00,50.00,50.00,50.00,50.00,65.96,69.65
4,85.53,63.79,50.00,50.00,50.00,83.46,87.61,50.00,50.00,50.00,56.86,56.57,55.83,84.25,87.93
5,50.00,50.00,72.39,65.17,83.46,50.00,50.00,81.00,77.63,77.93,56.60,56.89,57.62,50.00,50.00
6,50.00,53.82,76.54,69.32,87.61,50.00,50.00,85.15,81.78,82.08,60.75,61.04,61.78,50.00,50.00
7,83.07,61.33,50.00,50.00,50.00,81.00,85.15,50.00,50.00,50.00,54.40,54.11,53.38,81.79,85.48
8,79.70,57.96,50.00,50.00,50.00,77.63,81.78,50.00,50.00,50.00,51.03,50.74,50.00,78.42,82.10
9,80.00,58.26,50.00,50.00,50.00,77.93,82.08,50.00,50.00,50.00,51.33,51.04,50.30,78.72,82.41


In [86]:
r = no_of_nodes
c = no_of_nodes
 
final_node_time_mat = [[0 for _ in range(c)] for _ in range(r)]
final_node_cost_mat = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        list_i = df_nn_cn["Centroid_Numbers"][i]
        list_j = df_nn_cn["Centroid_Numbers"][j]

        sum_cost = 0
        sum_time = 0
        counter = 0
        for a in list_i:
            for b in list_j:
                sum_time = sum_time + df_motor_travel_time_mat[a][b]
                sum_cost = sum_cost + df_motor_travel_cost_mat[a][b]
                counter = counter + 1
        
        final_node_time_mat[i][j] = round(sum_time/counter,2)
        final_node_cost_mat[i][j] = round(sum_cost/counter,2)

print(f'travel time matrix of dimension {r} x {c} is {final_node_time_mat}')
print(f'travel cost matrix of dimension {r} x {c} is {final_node_cost_mat}')

travel time matrix of dimension 10 x 10 is [[0.0, 2.92, 1.66, 6.88, 2.23, 6.66, 5.75, 5.12, 7.38, 6.57], [2.92, 0.87, 4.59, 9.8, 4.95, 6.66, 2.82, 2.2, 10.3, 9.5], [1.66, 4.59, 0.0, 5.22, 2.79, 6.66, 7.41, 6.78, 5.72, 4.91], [6.88, 9.8, 5.22, 0.0, 4.86, 6.66, 12.63, 12.0, 0.5, 0.31], [2.23, 4.95, 2.79, 4.86, 2.95, 6.66, 7.77, 7.14, 5.36, 4.55], [6.66, 6.66, 6.66, 6.66, 6.66, 6.66, 6.95, 6.66, 6.66, 6.66], [5.75, 2.82, 7.41, 12.63, 7.77, 6.95, 0.7, 0.7, 13.13, 12.32], [5.12, 2.2, 6.78, 12.0, 7.14, 6.66, 0.7, 0.0, 12.5, 11.69], [7.38, 10.3, 5.72, 0.5, 5.36, 6.66, 13.13, 12.5, 0.0, 0.81], [6.57, 9.5, 4.91, 0.31, 4.55, 6.66, 12.32, 11.69, 0.81, 0.0]]
travel cost matrix of dimension 10 x 10 is [[50.0, 50.0, 50.0, 58.67, 52.2, 57.74, 53.94, 51.33, 60.75, 57.39], [50.0, 50.0, 51.36, 70.85, 56.26, 61.63, 50.0, 50.0, 72.93, 69.57], [50.0, 51.36, 50.0, 51.74, 50.0, 57.74, 60.88, 58.26, 53.82, 50.46], [58.67, 70.85, 51.74, 50.0, 56.22, 66.54, 82.62, 80.0, 50.0, 50.0], [52.2, 56.26, 50.0, 56.22, 5

In [87]:
motor_travel_time_mat = pd.DataFrame(final_node_time_mat)
motor_travel_time_mat

,0,1,2,3,4,5,6,7,8,9
0,0.00,2.92,1.66,6.88,2.23,6.66,5.75,5.12,7.38,6.57
1,2.92,0.87,4.59,9.80,4.95,6.66,2.82,2.20,10.30,9.50
2,1.66,4.59,0.00,5.22,2.79,6.66,7.41,6.78,5.72,4.91
3,6.88,9.80,5.22,0.00,4.86,6.66,12.63,12.00,0.50,0.31
4,2.23,4.95,2.79,4.86,2.95,6.66,7.77,7.14,5.36,4.55
5,6.66,6.66,6.66,6.66,6.66,6.66,6.95,6.66,6.66,6.66
6,5.75,2.82,7.41,12.63,7.77,6.95,0.70,0.70,13.13,12.32
7,5.12,2.20,6.78,12.00,7.14,6.66,0.70,0.00,12.50,11.69
8,7.38,10.30,5.72,0.50,5.36,6.66,13.13,12.50,0.00,0.81
9,6.57,9.50,4.91,0.31,4.55,6.66,12.32,11.69,0.81,0.00


In [88]:
motor_travel_cost_mat = pd.DataFrame(final_node_cost_mat)
motor_travel_cost_mat

,0,1,2,3,4,5,6,7,8,9
0,50.00,50.00,50.00,58.67,52.20,57.74,53.94,51.33,60.75,57.39
1,50.00,50.00,51.36,70.85,56.26,61.63,50.00,50.00,72.93,69.57
2,50.00,51.36,50.00,51.74,50.00,57.74,60.88,58.26,53.82,50.46
3,58.67,70.85,51.74,50.00,56.22,66.54,82.62,80.00,50.00,50.00
4,52.20,56.26,50.00,56.22,53.22,60.33,62.37,59.76,57.61,55.36
5,57.74,61.63,57.74,66.54,60.33,67.74,67.51,66.20,67.58,65.90
6,53.94,50.00,60.88,82.62,62.37,67.51,50.00,50.00,84.70,81.34
7,51.33,50.00,58.26,80.00,59.76,66.20,50.00,50.00,82.08,78.72
8,60.75,72.93,53.82,50.00,57.61,67.58,84.70,82.08,50.00,50.00
9,57.39,69.57,50.46,50.00,55.36,65.90,81.34,78.72,50.00,50.00


#### 9.1.d. Utility Functions

In [112]:
def U_car(travel_time, travel_cost):
    u_car = 0.345505 -0.060054*travel_time - 0.043648*travel_cost
    return u_car

def U_bus(travel_time, travel_cost):
    u_bus = 0.645505 -0.060054*travel_time - 0.043648*travel_cost
    return u_bus

def U_motor(travel_time, travel_cost):
    u_motor = 1.23213 -0.060054*travel_time - 0.043648*travel_cost
    return u_motor

In [113]:
r = no_of_nodes
c = no_of_nodes
 
utility_mat_bus = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        travel_time = bus_travel_time_mat[i][j]
        travel_cost = bus_travel_cost_mat[i][j]
        utility = U_bus(travel_time, travel_cost)
        utility_mat_bus[i][j] = round(utility, 4)

#print(f'travel time matrix of dimension {r} x {c} is {utility_mat_bus}')

df_utility_mat_bus = pd.DataFrame(utility_mat_bus)
df_utility_mat_bus

,0,1,2,3,4,5,6,7,8,9
0,-0.0092,-0.6839,-0.1371,-0.3906,-0.1600,-0.6284,-0.9266,-0.7367,-0.4008,-0.3137
1,-0.6839,-0.1834,-0.7915,-0.7176,-0.5414,-0.2200,-0.5341,-0.4158,-0.2626,-0.6176
2,-0.1371,-0.7915,-0.0092,-0.4728,-0.2729,-0.7217,-1.0586,-0.8687,-0.4830,-0.3960
3,-0.3906,-0.7176,-0.4728,-0.0092,-0.2398,-0.6548,-0.9681,-0.7782,-0.4266,-0.0861
4,-0.1600,-0.5414,-0.2729,-0.2398,-0.0747,-0.5143,-0.7474,-0.6200,-0.2867,-0.1636
5,-0.6284,-0.2200,-0.7217,-0.6548,-0.5143,-0.1365,-0.4134,-0.2951,-0.2374,-0.5779
6,-0.9266,-0.5341,-1.0586,-0.9681,-0.7474,-0.4134,-0.2777,-0.2777,-0.4608,-0.8456
7,-0.7367,-0.4158,-0.8687,-0.7782,-0.6200,-0.2951,-0.2777,-0.0092,-0.3425,-0.6830
8,-0.4008,-0.2626,-0.4830,-0.4266,-0.2867,-0.2374,-0.4608,-0.3425,-0.0092,-0.3503
9,-0.3137,-0.6176,-0.3960,-0.0861,-0.1636,-0.5779,-0.8456,-0.6830,-0.3503,-0.0092


In [114]:
utility_mat_car = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        travel_time = car_travel_time_mat[i][j]
        travel_cost = car_travel_cost_mat[i][j]
        utility = U_car(travel_time, travel_cost)
        utility_mat_car[i][j] = round(utility, 4)

#print(f'travel time matrix of dimension {r} x {c} is {utility_mat_car}')

df_utility_mat_car = pd.DataFrame(utility_mat_car)
df_utility_mat_car

,0,1,2,3,4,5,6,7,8,9
0,-1.4004,-2.3732,-1.9540,-3.6905,-2.1437,-3.6166,-3.3136,-3.1042,-3.8571,-3.5880
1,-2.3732,-1.6892,-2.9275,-4.6637,-3.0468,-3.6166,-2.3397,-2.1314,-4.8299,-4.5614
2,-1.9540,-2.9275,-1.4004,-3.1373,-2.3283,-3.6166,-3.8672,-3.6578,-3.3035,-3.0344
3,-3.6905,-4.6637,-3.1373,-1.4004,-3.0174,-3.6166,-5.6036,-5.3947,-1.5666,-1.5028
4,-2.1437,-3.0468,-2.3283,-3.0174,-2.3816,-3.6166,-3.9866,-3.7776,-3.1836,-2.9150
5,-3.6166,-3.6166,-3.6166,-3.6166,-3.6166,-3.6166,-3.7143,-3.6166,-3.6166,-3.6161
6,-3.3136,-2.3397,-3.8672,-5.6036,-3.9866,-3.7143,-1.6336,-1.6332,-5.7698,-5.5012
7,-3.1042,-2.1314,-3.6578,-5.3947,-3.7776,-3.6166,-1.6332,-1.4004,-5.5609,-5.2922
8,-3.8571,-4.8299,-3.3035,-1.5666,-3.1836,-3.6166,-5.7698,-5.5609,-1.4004,-1.6695
9,-3.5880,-4.5614,-3.0344,-1.5028,-2.9150,-3.6161,-5.5012,-5.2922,-1.6695,-1.4004


In [115]:
utility_mat_motor = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        travel_time = motor_travel_time_mat[i][j]
        travel_cost = motor_travel_cost_mat[i][j]
        utility = U_motor(travel_time, travel_cost)
        utility_mat_motor[i][j] = round(utility, 4)

#print(f'travel time matrix of dimension {r} x {c} is {utility_mat_motor}')

df_utility_mat_motor = pd.DataFrame(utility_mat_motor)
df_utility_mat_motor

,0,1,2,3,4,5,6,7,8,9
0,-0.9503,-1.1256,-1.0500,-1.7419,-1.1802,-1.6881,-1.4676,-1.3158,-1.8627,-1.6674
1,-1.1256,-1.0025,-1.2853,-2.4489,-1.5208,-1.8579,-1.1196,-1.0824,-2.5697,-2.3750
2,-1.0500,-1.2853,-0.9503,-1.3397,-1.1178,-1.6881,-1.8702,-1.7180,-1.4605,-1.2652
3,-1.7419,-2.4489,-1.3397,-0.9503,-1.5136,-2.0722,-3.1325,-2.9804,-0.9803,-0.9689
4,-1.1802,-1.5208,-1.1178,-1.5136,-1.2680,-1.8011,-1.9568,-1.8051,-1.6043,-1.4575
5,-1.6881,-1.8579,-1.6881,-2.0722,-1.8011,-2.1245,-2.1319,-2.0573,-2.1176,-2.0442
6,-1.4676,-1.1196,-1.8702,-3.1325,-1.9568,-2.1319,-0.9923,-0.9923,-3.2534,-3.0581
7,-1.3158,-1.0824,-1.7180,-2.9804,-1.8051,-2.0573,-0.9923,-0.9503,-3.1012,-2.9059
8,-1.8627,-2.5697,-1.4605,-0.9803,-1.6043,-2.1176,-3.2534,-3.1012,-0.9503,-0.9989
9,-1.6674,-2.3750,-1.2652,-0.9689,-1.4575,-2.0442,-3.0581,-2.9059,-0.9989,-0.9503


#### 9.2 Probability Matrices

In [116]:
def P_car(u_car, u_bus, u_motor):
    p_car = np.exp(u_car) / (np.exp(u_car) + np.exp(u_bus) + np.exp(u_motor) )
    return p_car

def P_bus(u_car, u_bus, u_motor):
    p_bus = np.exp(u_bus) / (np.exp(u_car) + np.exp(u_bus) + np.exp(u_motor) )
    return p_bus

def P_motor(u_car, u_bus, u_motor):
    p_motor = np.exp(u_motor) / (np.exp(u_car) + np.exp(u_bus) + np.exp(u_motor) )
    return p_motor

In [117]:
r = no_of_nodes
c = no_of_nodes
 
probability_mat_bus = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        u_car = df_utility_mat_car[i][j]
        u_bus = df_utility_mat_bus[i][j]
        u_motor = df_utility_mat_motor[i][j]
        prob = P_bus(u_car, u_bus, u_motor)
        probability_mat_bus[i][j] = round(prob, 4)

#print(f'travel time matrix of dimension {r} x {c} is {probability_mat_bus}')

df_probability_mat_bus = pd.DataFrame(probability_mat_bus)
df_probability_mat_bus

,0,1,2,3,4,5,6,7,8,9
0,0.6101,0.5472,0.6394,0.7717,0.6675,0.7159,0.5973,0.6046,0.7916,0.7715
1,0.5472,0.6014,0.5786,0.8359,0.6863,0.8144,0.5810,0.5906,0.9010,0.8390
2,0.6394,0.5786,0.6101,0.6712,0.6420,0.6965,0.6647,0.6715,0.6965,0.6708
3,0.7717,0.8359,0.6712,0.6101,0.7452,0.7728,0.8893,0.8925,0.5278,0.6038
4,0.6675,0.6863,0.6420,0.7452,0.7129,0.7569,0.7476,0.7417,0.7559,0.7474
5,0.7159,0.8144,0.6965,0.7728,0.7569,0.8563,0.8222,0.8280,0.8427,0.7820
6,0.5973,0.5810,0.6647,0.8893,0.7476,0.8222,0.5724,0.5723,0.9379,0.8937
7,0.6046,0.5906,0.6715,0.8925,0.7417,0.8280,0.5723,0.6101,0.9356,0.8943
8,0.7916,0.9010,0.6965,0.5278,0.7559,0.8427,0.9379,0.9356,0.6101,0.5586
9,0.7715,0.8390,0.6708,0.6038,0.7474,0.7820,0.8937,0.8943,0.5586,0.6101


In [118]:
probability_mat_car = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        u_car = df_utility_mat_car[i][j]
        u_bus = df_utility_mat_bus[i][j]
        u_motor = df_utility_mat_motor[i][j]
        prob = P_car(u_car, u_bus, u_motor)
        probability_mat_car[i][j] = round(prob, 4)

#print(f'travel time matrix of dimension {r} x {c} is {probability_mat_car}')

df_probability_mat_car = pd.DataFrame(probability_mat_car)
df_probability_mat_car

,0,1,2,3,4,5,6,7,8,9
0,0.1518,0.1010,0.1039,0.0285,0.0918,0.0361,0.0549,0.0567,0.0250,0.0292
1,0.1010,0.1334,0.0683,0.0162,0.0560,0.0273,0.0955,0.1062,0.0094,0.0163
2,0.1039,0.0683,0.1518,0.0467,0.0822,0.0385,0.0401,0.0413,0.0415,0.0479
3,0.0285,0.0162,0.0467,0.1518,0.0463,0.0400,0.0086,0.0088,0.1688,0.1464
4,0.0918,0.0560,0.0822,0.0463,0.0710,0.0340,0.0293,0.0315,0.0417,0.0477
5,0.0361,0.0273,0.0385,0.0400,0.0340,0.0264,0.0303,0.0299,0.0287,0.0375
6,0.0549,0.0955,0.0401,0.0086,0.0293,0.0303,0.1475,0.1476,0.0046,0.0085
7,0.0567,0.1062,0.0413,0.0088,0.0315,0.0299,0.1476,0.1518,0.0051,0.0089
8,0.0250,0.0094,0.0415,0.1688,0.0417,0.0287,0.0046,0.0051,0.1518,0.1493
9,0.0292,0.0163,0.0479,0.1464,0.0477,0.0375,0.0085,0.0089,0.1493,0.1518


In [119]:
probability_mat_motor = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        u_car = df_utility_mat_car[i][j]
        u_bus = df_utility_mat_bus[i][j]
        u_motor = df_utility_mat_motor[i][j]
        prob = P_motor(u_car, u_bus, u_motor)
        probability_mat_motor[i][j] = round(prob, 4)

#print(f'travel time matrix of dimension {r} x {c} is {probability_mat_bus}')

df_probability_mat_motor = pd.DataFrame(probability_mat_motor)
df_probability_mat_motor

,0,1,2,3,4,5,6,7,8,9
0,0.2381,0.3518,0.2566,0.1998,0.2407,0.2481,0.3478,0.3388,0.1835,0.1993
1,0.3518,0.2651,0.3531,0.1480,0.2577,0.1583,0.3235,0.3032,0.0897,0.1447
2,0.2566,0.3531,0.2381,0.2821,0.2758,0.2650,0.2952,0.2872,0.2620,0.2813
3,0.1998,0.1480,0.2821,0.2381,0.2085,0.1873,0.1021,0.0987,0.3034,0.2498
4,0.2407,0.2577,0.2758,0.2085,0.2162,0.2090,0.2231,0.2268,0.2024,0.2049
5,0.2481,0.1583,0.2650,0.1873,0.2090,0.1173,0.1475,0.1421,0.1286,0.1805
6,0.3478,0.3235,0.2952,0.1021,0.2231,0.1475,0.2801,0.2801,0.0575,0.0978
7,0.3388,0.3032,0.2872,0.0987,0.2268,0.1421,0.2801,0.2381,0.0593,0.0968
8,0.1835,0.0897,0.2620,0.3034,0.2024,0.1286,0.0575,0.0593,0.2381,0.2920
9,0.1993,0.1447,0.2813,0.2498,0.2049,0.1805,0.0978,0.0968,0.2920,0.2381


#### 10. Modal Share Matrix for Bus

In [128]:
# Import the Readjusted Final Trip Distribution from Matlab 
final_trip_dsbn = pd.read_csv("final_trip_distribution_mat.csv", header=None)

r = no_of_nodes
c = no_of_nodes

for i in range(0,r):
    for j in range(0,c):
        a = final_trip_dsbn[i][j]
        final_trip_dsbn.loc[i,j] = int(a)
        
final_trip_dsbn

,0,1,2,3,4,5,6,7,8,9
0,1321.0,1978.0,1104.0,793.0,2623.0,2567.0,2232.0,1113.0,1999.0,428.0
1,1978.0,9281.0,871.0,1048.0,2984.0,10859.0,8123.0,4053.0,5389.0,566.0
2,1104.0,871.0,972.0,492.0,1547.0,1592.0,1384.0,690.0,1240.0,265.0
3,793.0,1048.0,492.0,721.0,1107.0,1186.0,1031.0,514.0,923.0,302.0
4,2623.0,2984.0,1547.0,1107.0,6472.0,6643.0,5776.0,2882.0,5173.0,1177.0
5,2567.0,10859.0,1592.0,1186.0,6643.0,6619.0,5264.0,2626.0,2981.0,313.0
6,2232.0,8123.0,1384.0,1031.0,5776.0,5264.0,4494.0,1840.0,1398.0,147.0
7,1113.0,4053.0,690.0,514.0,2882.0,2626.0,1840.0,1619.0,958.0,100.0
8,1999.0,5389.0,1240.0,923.0,5173.0,2981.0,1398.0,958.0,788.0,82.0
9,428.0,566.0,265.0,302.0,1177.0,313.0,147.0,100.0,82.0,151.0


In [131]:
modal_share_mat_bus = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        trip_bus = final_trip_dsbn[i][j]
        prob_bus = df_probability_mat_bus[i][j]
        modal_share_mat_bus[i][j] = round(trip_bus*prob_bus, 0)

#print(f'travel time matrix of dimension {r} x {c} is {modal_share_mat_bus}')

df_modal_share_mat_bus = pd.DataFrame(modal_share_mat_bus)
df_modal_share_mat_bus

,0,1,2,3,4,5,6,7,8,9
0,806.0,1082.0,706.0,612.0,1751.0,1838.0,1333.0,673.0,1582.0,330.0
1,1082.0,5582.0,504.0,876.0,2048.0,8844.0,4719.0,2394.0,4855.0,475.0
2,706.0,504.0,593.0,330.0,993.0,1109.0,920.0,463.0,864.0,178.0
3,612.0,876.0,330.0,440.0,825.0,917.0,917.0,459.0,487.0,182.0
4,1751.0,2048.0,993.0,825.0,4614.0,5028.0,4318.0,2138.0,3910.0,880.0
5,1838.0,8844.0,1109.0,917.0,5028.0,5668.0,4328.0,2174.0,2512.0,245.0
6,1333.0,4719.0,920.0,917.0,4318.0,4328.0,2572.0,1053.0,1311.0,131.0
7,673.0,2394.0,463.0,459.0,2138.0,2174.0,1053.0,988.0,896.0,89.0
8,1582.0,4855.0,864.0,487.0,3910.0,2512.0,1311.0,896.0,481.0,46.0
9,330.0,475.0,178.0,182.0,880.0,245.0,131.0,89.0,46.0,92.0
